In [7]:
import pandas as pd

In [8]:
df = pd.read_csv(r'c:\Users\SAURAV RAJ\Desktop\dataset\spotify_millsongdata.csv')

In [9]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [10]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [11]:
df.shape

(57650, 4)

In [12]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [13]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [14]:
df.head(10)

,artist,song,text
0,Cat Stevens,Tuesday's Dead,"If I make a mark in time, I can't say the mark..."
1,New Order,Who's Joe?,Hey Joe what you doing? \r\nIt seems like it'...
2,Ray Boltz,Sent By The Father,One more time you've heard the story \r\nOf o...
3,Electric Light Orchestra,Is It Alright,Dear Jo: \r\nI don't know where to begin so I...
4,Alan Parsons Project,You're Gonna Get Your Fingers Burned,Watch me closely catch my eye \r\nIf you do I...
5,Adam Sandler,Retarded Burger King Song,I work at Burger King making flame-broiled who...
6,Leann Rimes,Fight,"Hush, Hush why don't you want to talk about it..."
7,Great Big Sea,Dream To Live,Set off for America far across the ragin' sea ...
8,Billy Joel,You Look So Good To Me,"Ah, you look so good to me \r\nWith my eyes o..."
9,Frank Zappa,"Concentration Moon, Part One","[Fillmore East, nyc \r\nJune 5-6, 1971] \r\n..."


In [15]:
df['text'][0]

"If I make a mark in time, I can't say the mark is mine.  \r\nI'm only the underline of the Word.  \r\nYes, I'm like him, just like you, I can't tell you what to do.  \r\nLike everybody else I'm searching through what I've heard.  \r\n  \r\n[Chorus]  \r\nWhoa, Where do you go?  \r\nWhen you don't want no one to know?  \r\nWho told tomorrow Tuesday's dead  \r\n  \r\nOh preacher won't you paint my dream, won't you show me where you've been  \r\nShow me what I haven't seen to ease my mind.  \r\nCause I will learn to understand, if I have a helping hand.  \r\nI wouldn't make another demand all my life.  \r\n  \r\n[Chorus]  \r\n  \r\nWhat's my sex, what's my name, all in all it's all the same.  \r\nEverybody plays a different game, that is all.  \r\nNow, man may live, man may die searching for the question why.  \r\nBut if he tries to rule the sky he must fall.  \r\n  \r\n[Chorus]  \r\n  \r\nNow every second on the nose, the humdrum of the city grows.  \r\nreaching out beyond the throes of 

In [16]:
# df = df.sample(5000)

In [17]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [18]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [21]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [31]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [30]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\SAURAV
[nltk_data]     RAJ\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [35]:
similarity[0]

array([1.        , 0.02417713, 0.09548384, ..., 0.03700717, 0.08539972,
       0.00914527])

In [36]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [39]:
def recommendation(song_df):
    if song_df not in df['song'].values:
        return f"'{song_df}' not found in dataset"
    
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

    songs = [df.iloc[m_id[0]].song for m_id in distances[1:21]]
    
    return songs


In [41]:
recommendation('Fight')

['She Wants You Back',
 "Heart's Breakin' All Over Town",
 'I Wanna Be With You',
 'Please Stay',
 'Sleep Tonight',
 "Don't You Want To Be There",
 "Who's Right, Who's Wrong",
 "I'd Still Want You",
 'A Woman, A Lover, A Friend',
 'Love Is All That Matters',
 "I Don't Want To Cry Anymore",
 'Deep In My Heart',
 'Right Now',
 'Steel Bar Blues',
 'You Know That I Want To',
 "What I Want Is What I've Got",
 "It Doesn't Matter",
 'Moment Of Truth',
 'Say It',
 'Lay Down Sally']

In [42]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))